## Importing necessary libraries

In [4]:
from collections import Counter
import pandas as pd
import numpy as np
import os
import re
import string
import pickle
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans

#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
remove_these = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
REMOVE_STOPWORDS = True

## Text Preprocessing
- Dataset is not preprocessed text preprocessing stage include preprocessing of the dataset

In [6]:
# Change this value if you want to train more or less data
NUM_DATA = 50000

In [9]:
# Add the location of Train data here
%%time

with open('Train_data.txt','r') as f:
    lines = f.readlines()
    words = lines[17].replace('%','').split(',')
    all_songs_dict = dict()
    for i,line in list(enumerate(lines))[18:NUM_DATA]:
        song_info = line.split(',')
        MSDID = song_info[0]
        song_bow = [x.split(':') for x in song_info[2:]]
        song_dict = {}
        for word, word_count in song_bow:
            song_dict[int(word)] = int(word_count.replace('\n',''))
        word_lists = [[words[word-1]]*song_dict[word] for word in song_dict.keys()]
        song = [word for word_list in word_lists for word in word_list]
        if REMOVE_STOPWORDS:
            song = [w for w in song if w not in remove_these]
        all_songs_dict[str(MSDID)] = ' '.join(song).replace('\n','')

ValueError: ignored

In [ ]:
print(len(all_songs_dict.keys()))
song_msd_ids = list(all_songs_dict.keys())

49982


In [ ]:
# Add the match dataset here
%%time

all_song_meta_dict = dict()
with open('match_data.txt','r') as f:
    lines = f.readlines()
    print(lines[18:25])
    for i in range(18, len(lines)):
      line = lines[i].split('<SEP>')
      MSDID = line[0]
      artist = line[1]
      title = line[2]
      all_song_meta_dict[str(MSDID)] = {'artist': artist, 'title': title}

['TRMMMKD128F425225D<SEP>Karkkiautomaatti<SEP>Tanssi vaan<SEP>4418550<SEP>Karkkiautomaatti<SEP>Tanssi vaan\n', 'TRMMMRX128F93187D9<SEP>Hudson Mohawke<SEP>No One Could Ever<SEP>8898149<SEP>Hudson Mohawke<SEP>No One Could Ever\n', 'TRMMMCH128F425532C<SEP>Yerba Brava<SEP>Si Vos Querés<SEP>9239868<SEP>Yerba Brava<SEP>Si vos queres\n', 'TRMMMXN128F42936A5<SEP>David Montgomery<SEP>Symphony No. 1 G minor "Sinfonie Serieuse"/Allegro con energia<SEP>5346741<SEP>Franz Berwald<SEP>Symphony No. 1 in G minor "Sinfonie Sérieuse": I. Allegro con energia\n', "TRMMMBB12903CB7D21<SEP>Kris Kross<SEP>2 Da Beat Ch'yall<SEP>2511405<SEP>Kris Kross<SEP>2 Da Beat Ch'yall\n", 'TRMMMHY12903CB53F1<SEP>Joseph Locke<SEP>Goodbye<SEP>793273<SEP>Joseph LoDuca<SEP>Goodbye\n', "TRMMMNS128F93548E1<SEP>3 Gars Su'l Sofa<SEP>L'antarctique<SEP>7503609<SEP>3 gars su'l sofa<SEP>L'Antarctique\n"]
CPU times: user 1.57 s, sys: 200 ms, total: 1.77 s
Wall time: 1.84 s


In [ ]:
len(all_song_meta_dict.keys())

779056

In [ ]:

data_dict = {
    'MSDID': list(all_songs_dict.keys()),
    'cleaned_text': [all_songs_dict[x] for x in all_songs_dict.keys()]
    }
msdid_df = pd.DataFrame.from_dict(data_dict)
print(msdid_df.shape)
msdid_df.head()

(49982, 2)


,MSDID,cleaned_text
0,TRAAAAV128F421A322,like like de got would seem someon understand ...
1,TRAAABD128F429CF47,know know know know know time time time la la ...
2,TRAAAED128E0783FAB,love love love love love love love love love l...
3,TRAAAEF128F4273421,know got got got feel let would would would ey...
4,TRAAAEW128F42930C0,like take would wo someth stay burn burn burn ...


In [ ]:
d = {
    'MSDID': msdid_df['MSDID'],
    'artist': [all_song_meta_dict[x]['artist'] for x in msdid_df['MSDID']],
    'title': [all_song_meta_dict[x]['title'] for x in msdid_df['MSDID']]
    }
meta_df = pd.DataFrame.from_dict(d)
print(meta_df.shape)
meta_df.head()

(49982, 3)


,MSDID,artist,title
0,TRAAAAV128F421A322,Western Addiction,A Poor Recipe For Civic Cohesion
1,TRAAABD128F429CF47,The Box Tops,Soul Deep
2,TRAAAED128E0783FAB,Jamie Cullum,It's About Time
3,TRAAAEF128F4273421,Adam Ant,Something Girls
4,TRAAAEW128F42930C0,Broken Spindles,Burn My Body (Album Version)


In [ ]:
master_df = pd.merge(msdid_df, meta_df, on='MSDID', how='left')
print(master_df.shape)
master_df.head()

(49982, 4)


,MSDID,cleaned_text,artist,title
0,TRAAAAV128F421A322,like like de got would seem someon understand ...,Western Addiction,A Poor Recipe For Civic Cohesion
1,TRAAABD128F429CF47,know know know know know time time time la la ...,The Box Tops,Soul Deep
2,TRAAAED128E0783FAB,love love love love love love love love love l...,Jamie Cullum,It's About Time
3,TRAAAEF128F4273421,know got got got feel let would would would ey...,Adam Ant,Something Girls
4,TRAAAEW128F42930C0,like take would wo someth stay burn burn burn ...,Broken Spindles,Burn My Body (Album Version)


In [ ]:
final_df = master_df.copy()
print(final_df.shape)
final_df.head()

(49982, 4)


,MSDID,cleaned_text,artist,title
0,TRAAAAV128F421A322,like like de got would seem someon understand ...,Western Addiction,A Poor Recipe For Civic Cohesion
1,TRAAABD128F429CF47,know know know know know time time time la la ...,The Box Tops,Soul Deep
2,TRAAAED128E0783FAB,love love love love love love love love love l...,Jamie Cullum,It's About Time
3,TRAAAEF128F4273421,know got got got feel let would would would ey...,Adam Ant,Something Girls
4,TRAAAEW128F42930C0,like take would wo someth stay burn burn burn ...,Broken Spindles,Burn My Body (Album Version)


### Get Embeddings
- Machine learning doesn't work on text, will represented the text to respective embeddings and vectors.


In [ ]:
%%time

# get training data
data = final_df['cleaned_text'].tolist()
# test_data = test_df['cleaned_text'].tolist()
train_corpus = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in list(enumerate(data))]
# build model
model = Doc2Vec(vector_size=50, min_count=1, epochs=10, dm=0)
model.build_vocab(train_corpus)
# train model
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
# Save the model
model.save('doc_vec_model')

In [ ]:
# Load the model
model = Doc2Vec.load('doc_vec_model')

In [ ]:
embeddings_df = pd.DataFrame([model.infer_vector(data) for data in data])

## Reducing the dimension
- Having 50 dimension for computing will be computational expensive so we reduce the dimension to 3
- For higher accuracy you have increase the dimension but which will effective the training time exponentionally

In [ ]:
# Scale the embeddings in between -1 and 1
scaler = StandardScaler()
embeddings_df = scaler.fit_transform(embeddings_df)

In [ ]:
# PCA to decrease the embedding dimension into 3 dimension
n=3
pca = PCA(n_components=n)
song_embedding = pca.fit_transform(embeddings_df)
pca_df = pd.DataFrame(data = song_embedding, columns = list(range(n)))
pca_df['title'] = final_df['title']
pca_df['artist'] = final_df['artist']

In [ ]:
pca_df

,0,1,2,title,artist
0,-0.550833,-2.566153,-1.101641,A Poor Recipe For Civic Cohesion,Western Addiction
1,0.323452,0.521762,3.454032,Soul Deep,The Box Tops
2,-0.072394,-0.974304,3.109484,It's About Time,Jamie Cullum
3,-1.058543,1.467625,1.425659,Something Girls,Adam Ant
4,-1.269913,-1.208941,-1.650759,Burn My Body (Album Version),Broken Spindles
...,...,...,...,...,...
49977,1.313978,-0.593696,-0.276774,Adeste Fideles,Cleopatra
49978,-1.101754,0.575205,0.618980,Say No More,Entity
49979,0.380788,-1.454965,0.122360,Stay In The Shade,José González
49980,-0.445382,0.629375,1.470369,Second Childhood,Foghat


In [ ]:
# Will save the laod as a pickle file
pickle.dump(scaler, open('scaler.pkl', 'wb'))
pickle.dump(pca, open('pca.pkl', 'wb'))

In [ ]:
# Drop artist column as there is no requirement for it
song_data = pca_df.drop('artist',axis=1)

## KMeans Clustering
- Will divide all the lyrics into clusters like a group whose lyrics have similar meaning

In [ ]:
y = song_data.pop('title')
X = song_data

In [ ]:
N_clutser = int(NUM_DATA * 0.01)

In [ ]:
N_clutser

500

In [ ]:
# Do a kmeans means cluster on N_CLusters
kmeans = KMeans(n_clusters = N_clutser, init='k-means++')
kmeans.fit(X)

KMeans(n_clusters=500)

In [ ]:
# Will save the kmeans model with 500 clusters
pickle.dump(kmeans, open('kmeans_model_500.pkl', 'wb'))

In [ ]:
df_labeled=pd.concat([X,pd.Series(kmeans.labels_,name='label'),y],axis=1)  #I take the initial dataset: 
df_labeled

,0,1,2,label,title
0,4.884261,22.625668,8.019410,433,A Poor Recipe For Civic Cohesion
1,15.171036,-0.334421,16.482397,275,Soul Deep
2,20.286266,-14.874681,-3.259438,376,It's About Time
3,11.932621,-13.828093,-6.590390,61,Something Girls
4,1.805672,14.741771,-0.578643,282,Burn My Body (Album Version)
...,...,...,...,...,...
49977,-15.918822,-13.302044,-6.218614,224,Adeste Fideles
49978,5.430110,-3.710590,4.790823,217,Say No More
49979,18.216553,4.193786,6.335515,340,Stay In The Shade
49980,-1.738006,-0.129301,2.068179,161,Second Childhood


In [ ]:
df_labeled['label'].value_counts()

20     253
487    220
209    205
33     193
165    192
      ... 
479     47
349     47
183     47
301     43
493     41
Name: label, Length: 500, dtype: int64

In [ ]:
# Save the dataframe with cluster label
df_labeled.to_csv('songs_selection_500.csv')

In [ ]:
#See the respective cluster for understanding
df_labeled[df_labeled['label']==0]

,0,1,2,label,title
2,36.728012,20.205135,-0.617097,0,It's About Time
3,16.921083,17.421444,-0.942628,0,Something Girls
12,19.396084,20.747402,-23.297571,0,Drown In My Own Tears (24-Bit Digitally Remast...
24,4.071661,8.328055,-5.027319,0,Caught In A Dream
40,-0.258732,38.281017,15.466446,0,They're Coming For You
...,...,...,...,...,...
9962,23.229683,16.001333,17.446787,0,...Baby One More Time
9963,15.912280,31.704798,25.555372,0,Love Like This
9967,0.004066,25.338612,5.636103,0,That´s All
9975,-11.609483,29.654091,0.255605,0,Worlds Apart
